In [1]:
import numpy as np
import random
import time
import pyaudio
import struct

In [2]:
CHUNK = 1024
FORMAT = pyaudio.paFloat32
CHANNELS = 1
RATE = 44100
RECORD_SECONDS = 1.01

p = pyaudio.PyAudio()

inputs = p.open(format=FORMAT,
                channels=CHANNELS,
                rate=RATE,
                input=True,
                frames_per_buffer=CHUNK)
outputs = p.open(format=FORMAT,
                channels=CHANNELS,
                rate=RATE,
                output=True,
                frames_per_buffer=CHUNK)

In [3]:
def play_sound(array):
    for i in range(len(array)):
        outputs.write(array[i])

In [4]:
test = np.load("./Act_train/activation/5.npy").flatten()
play_sound(test)

In [5]:
test = np.load("./Act_train/negative/30.npy").flatten()
play_sound(test)

In [6]:
act = []
for i in range(15):
    act.append(np.frombuffer(np.load("./Act_train/activation/"+str(i+1)+".npy").flatten()))

In [7]:
neg = []
for i in range(120):
    neg.append(np.frombuffer(np.load("./Act_train/negative/"+str(i+1)+".npy").flatten()))

In [8]:
act, neg = np.array(act), np.array(neg)
print(act.shape)
print(neg.shape)
print(act[0].shape)
print(neg[0].shape)

(15, 22016)
(120, 22016)
(22016,)
(22016,)


In [9]:
Tx = 1376 # The number of time steps input to the model from the spectrogram
n_freq = 16

In [10]:
def create_training_example(activates, negatives):
    """
    Creates a training example with a given background, activates, and negatives.
    
    Arguments:
    background -- a 10 second background audio recording
    activates -- a list of audio segments of the word "activate"
    negatives -- a list of audio segments of random words that are not "activate"
    Ty -- The number of time steps in the output

    Returns:
    x -- the spectrogram of the training example
    y -- the label at each time step of the spectrogram
    """

    y = 0
    x = None
    flag = np.random.randint(1, 121)
    if flag%5==0:
        x = activates[np.random.choice(len(activates))]
        y = 1
    else:
        x = negatives[np.random.choice(len(negatives))]
    
    return x, y

In [ ]:
#print(len(act[0]))
#graph_spectrogram(act[0])

In [ ]:
x, y = create_training_example(act, neg)
print(type(x))
print(type(x[0]))
print(type(y))
print(y)
print(x)

In [ ]:
nsamples = 3000

In [ ]:
nsamples = 3000
X = []
Y = []
for i in range(0, nsamples):
    if i%100 == 0:
        print(i)
    x, y = create_training_example(act, neg)
    X.append(x)
    Y.append(y)
X = np.array(X)
Y = np.array(Y).reshape((nsamples, 1))

In [12]:
print(X.shape)
print(Y.shape)
print("True" if 1 in Y else "False")

(3000, 22016)
(3000, 1)
True


In [10]:
# Save the data for further uses
#np.save(f'./Act_train/XY_train/X.npy', X)
#np.save(f'./Act_train/XY_train/Y.npy', Y)
# Load the preprocessed training examples
X = np.load("./Act_train/XY_train/X.npy")
Y = np.load("./Act_train/XY_train/Y.npy")

In [ ]:
X_dev = []
Y_dev = []
for i in range(0, nsamples):
    if i%100 == 0:
        print(i)
    x, y = create_training_example(act, neg)
    X_dev.append(x)
    Y_dev.append(y)
X_dev = np.array(X_dev)
Y_dev = np.array(Y_dev).reshape((nsamples, 1))
np.save(f'./Act_train/XY_dev/X_dev.npy', X_dev)
np.save(f'./Act_train/XY_dev/Y_dev.npy', Y_dev)

In [ ]:
X_dev = []
Y_dev = []

In [26]:
X_dev = np.load("./Act_train/XY_dev/X_dev.npy")
Y_dev = np.load("./Act_train/XY_dev/Y_dev.npy")

In [38]:
print((Y_dev[0]))
print(type(Y_dev))
print(Y_dev.shape)
print(X_dev.shape)


[1]
<class 'numpy.ndarray'>
(3000, 1)
(3000, 22016)


In [13]:
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import Model, load_model, Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, Input, Masking, TimeDistributed, LSTM, Conv1D
from tensorflow.keras.layers import GRU, Bidirectional, BatchNormalization, Reshape, Flatten
from tensorflow.keras.optimizers import Adam

In [ ]:
model = load_model("modelo")

In [19]:
def modelf(input_shape):
    """
    Function creating the model's graph in Keras.
    
    Argument:
    input_shape -- shape of the model's input data (using Keras conventions)

    Returns:
    model -- Keras model instance
    """
    
    X_input = Input(shape = input_shape)
    
    X = Reshape((Tx,n_freq))(X_input)
    X = Conv1D(filters = 196, kernel_size=15, strides=4)(X)
    X = BatchNormalization()(X)
    X = Activation("relu")(X)
    X = Dropout(rate=0.8)(X)                                  

    X = GRU(units = 128, return_sequences=True)(X)
    X = Dropout(rate = 0.8)(X)
    X = BatchNormalization()(X)                           
    
    X = GRU(units = 128, return_sequences=True)(X)
    X = Dropout(rate = 0.8)(X)       
    X = BatchNormalization()(X) 

    X = GRU(units = 60, return_sequences=True)(X)
    X = Dropout(rate = 0.8)(X)       
    X = BatchNormalization()(X)
    
    X = GRU(units = 60, return_sequences=True)(X)
    X = Dropout(rate = 0.8)(X)       
    X = BatchNormalization()(X) 
    
    X = TimeDistributed(Dense(1, activation = "sigmoid"))(X) 
    X = Dense(1, activation = "sigmoid")(Flatten()(X))

    model = Model(inputs = X_input, outputs = X)
    
    return model

In [20]:
model = modelf(input_shape = (X.shape[1]))

In [21]:
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 22016)]           0         
                                                                 
 reshape_1 (Reshape)         (None, 1376, 16)          0         
                                                                 
 conv1d_1 (Conv1D)           (None, 341, 196)          47236     
                                                                 
 batch_normalization_4 (Batc  (None, 341, 196)         784       
 hNormalization)                                                 
                                                                 
 activation_1 (Activation)   (None, 341, 196)          0         
                                                                 
 dropout_4 (Dropout)         (None, 341, 196)          0         
                                                           

In [22]:
opt = Adam(learning_rate=1e-6, beta_1=0.9, beta_2=0.999)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=["accuracy"])

In [23]:
model.fit(X, Y, batch_size = 16, epochs=50)

Epoch 1/50
188/188 [==============================] - 80s 394ms/step - loss: 0.6657 - accuracy: 0.6097
Epoch 2/50
188/188 [==============================] - 73s 388ms/step - loss: 0.6563 - accuracy: 0.6170
Epoch 3/50
188/188 [==============================] - 74s 392ms/step - loss: 0.6550 - accuracy: 0.6287
Epoch 4/50
188/188 [==============================] - 73s 391ms/step - loss: 0.6395 - accuracy: 0.6517
Epoch 5/50
188/188 [==============================] - 74s 391ms/step - loss: 0.6357 - accuracy: 0.6713
Epoch 6/50
188/188 [==============================] - 73s 387ms/step - loss: 0.6315 - accuracy: 0.6853
Epoch 7/50
188/188 [==============================] - 73s 390ms/step - loss: 0.6204 - accuracy: 0.7043
Epoch 8/50
188/188 [==============================] - 73s 389ms/step - loss: 0.6169 - accuracy: 0.7030
Epoch 9/50
188/188 [==============================] - 73s 390ms/step - loss: 0.6107 - accuracy: 0.7123
Epoch 10/50
188/188 [==============================] - 73s 388ms/step - l

In [27]:
loss, acc, = model.evaluate(X_dev, Y_dev)
print("Dev set accuracy = ", acc)

94/94 [==============================] - 15s 145ms/step - loss: 0.4955 - accuracy: 0.8057
Dev set accuracy =  0.8056666851043701


In [25]:
X, Y= [], []

In [ ]:
model.save("modelo")

In [32]:
frames = []
for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
        data = inputs.read(CHUNK)
        frames.append(data)
aux = np.frombuffer(np.array(frames))

In [33]:
predictions = model.predict(aux.reshape((1,aux.shape[0])))

1/1 [==============================] - 0s 52ms/step


In [34]:
1-predictions

array([[0.7733404]], dtype=float32)